# I/O操作

In [5]:
import faiss
# faiss.write_index(index, "index_file.index") #将index保存为index_file.index文件
index = faiss.read_index("index_file.index") # 读入index_file.index文件

In [3]:
#完全复制一个index
index_new = faiss.clone_index(index)
index_cpu_to_gpu = faiss.index_cpu_to_gpu()
#index_cpu_to_gpu
#todo

NotImplementedError: Wrong number or type of arguments for overloaded function 'index_cpu_to_gpu'.
  Possible C/C++ prototypes are:
    faiss::gpu::index_cpu_to_gpu(faiss::gpu::GpuResources *,int,faiss::Index const *,faiss::gpu::GpuClonerOptions const *)
    faiss::gpu::index_cpu_to_gpu(faiss::gpu::GpuResources *,int,faiss::Index const *)


# Index factory
用一个字符串构建Index，用逗号分割可以分为3部分：1.前处理部分；2.倒排表（聚类）；3.细化后处理部分  

在前处理部分（preprocessing）：  
    1.PCA。"PCA64"表示通过PCA将数据维度降为64，"PCAR64"表示增加了随机旋转（random rotation）。  
    2.OPQ。"OPQ16"表示用OPQMatrix将数组量化为16位（待完善）  

倒排表部分（inverted file）：  
    1."IVF4096"表示建立一个大小是4096的倒排表，即聚类为4096类。
    
细化部分（refinement）：  
    1."Flat"保存完整向量，通过IndexFlat或者IndexIVFFlat实现；  
    2."PQ16"将向量编码为16byte，通过IndexPQ或者IndexIVFPQ实现；

In [6]:
index = faiss.index_factory(128, "PCA80,Flat")  # 原始向量128维，用PCA降为80维，然后应用精确搜索
index = faiss.index_factory(128, "OPQ16_64,IMI2x8,PQ8+16")  # 原始向量128维，用OPQ降为64维，分为16类，用2*8bit的倒排多索引，用PQ编码为8byte保存，检索时使用16byte